In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_2 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchains.

In [3]:
stanmodel = Stanmodel(name="m12.6.2",  model=m12_6_2,
output_format=:mcmcchains);

Input data for cmdstan

In [4]:
m12_6_2_data = Dict("N" => size(d, 1), "T" => d[:total_tools],
"N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_2_data, ProjDir,
diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StanModels/notebooks/12/tmp/m12.6.2.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters    │ mean       │ std       │ naive_se   │ mcse       │
│     │ Symbol        │ Float64    │ Float64   │ Float64    │ Float64    │
├─────┼───────────────┼────────────┼───────────┼────────────┼────────────┤
│ 1   │ a_society.1   │ -0.213349  │ 0.245754  │ 0.00388571 │ 0.00578208 │
│ 2   │ a_society.2   │ 0.0376357  │ 0.223694  │ 0.00353691 │ 0.00586615 │
│ 3   │ a_society.3   │ -0.0533204 │ 0.194623  │ 0.00307726 │ 0.00450923 │
│ 4   │ a_society.4   │ 0.319279   │ 0.189459  │ 0.00299562 │ 0.00498301 │
│ 5   │ a_society.5   │ 0.0389642  │ 0.180694  │ 0.00285702 │ 0.00454844 │
│ 6   │ a_society.6   │ -0.327294  │ 0.215     │ 0.00339945 │ 0.00562222 │
│ 7   │ a_society.7   │ 0.142341   │ 0.175132  │ 0.00276908 │ 0.004958   │
│ 8   │ a_society.8   │ -0.175516  │ 0.185869  │ 0.00293885 │ 0.00494997 │
│ 9   │ a_society.9   │ 0.272536   │ 0.179362  │ 0.00283596 │ 0.004837   │
│ 10  │ a_soci

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*